In [1]:
import data
import glob
import matplotlib.pyplot as plt
import models
import numpy as np
import os
import pandas as pd
import pypickle
import strategy
import time

from data import * 
from feature_engineering import FeaturesEngineering
from models import Models
from io import StringIO
from IPython.display import display
from sklearn.linear_model import LassoCV
from utils import log

2025-03-01 17:36:20.094396: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-01 17:36:20.104648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740821780.116926   21418 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740821780.120735   21418 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-01 17:36:20.133402: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
models = Models(use_xgboost_gpu=True)

# Initializing the FeaturesEngineering obj
feat = FeaturesEngineering()
# Default config
feat.default_config

dict_data = {}
dict_cv_features = {}

In [3]:
dir_project = os.getcwd()
dir_models = os.path.join(dir_project, 'models')
dir_models_base = os.path.join(dir_project, 'models', 'base')
dir_models_scaler = os.path.join(dir_project, 'models', 'scaler')
dir_models_best = os.path.join(dir_project, 'models', 'best')

print(f"""
Project directory: {dir_project}
Models directory: {dir_models}
Base models directory: {dir_models_base}
Scalers directory: {dir_models_scaler}
Best models directory: {dir_models_best}
""")


Project directory: /home/alfred/projects/mscfe-masters-project/src
Models directory: /home/alfred/projects/mscfe-masters-project/src/models
Base models directory: /home/alfred/projects/mscfe-masters-project/src/models/base
Scalers directory: /home/alfred/projects/mscfe-masters-project/src/models/scaler
Best models directory: /home/alfred/projects/mscfe-masters-project/src/models/best



# Read raw data

In [4]:
# Define S3 bucket and file path
bucket_name = 'mscfe-capstone-project-bucket'
file_key = 'filtered_data.csv'

filtered = pd.read_csv('filtered_data.csv').drop(['Unnamed: 0'], axis=1)
display(filtered)

,seriesX,seriesY,correlation,spread_adf_stat,spread_adf_pValue,spread_stationary,series_coint_stat,series_coint_pValue,series_cointegrated
0,GOOG,GOOGL,0.999931,-4.971937,0.000025,True,-4.976339,0.000182,True
1,META,DAXEX_ETF,0.888934,-2.917056,0.043400,True,-3.483091,0.033924,True
2,AMZN,BNBUSDT,0.877059,-3.631203,0.005192,True,-3.545100,0.028601,True
3,AMZN,TRXUSDT,0.814577,-3.869803,0.002269,True,-3.387413,0.043768,True
4,SPY_ETF,AVAXUSDT,0.847292,-3.309927,0.014441,True,-3.698512,0.018406,True
5,SPY_ETF,ETHUSDT,0.862483,-3.428205,0.010034,True,-3.737367,0.016394,True
6,EQQQ_ETF,BNBUSDT,0.886060,-3.816404,0.002744,True,-3.670509,0.019987,True
7,BNBUSDT,ADAUSDT,0.901459,-3.279504,0.015820,True,-3.775562,0.014607,True
8,BNBUSDT,BTCUSDT,0.884604,-2.909961,0.044202,True,-3.393160,0.043116,True
9,BNBUSDT,TRXUSDT,0.933562,-3.656380,0.004770,True,-4.003675,0.007095,True


In [5]:
print('********************')
pairs = filtered['seriesX'].iloc[1:] + ' ' + filtered['seriesY'].iloc[1:]
print('Trading pairs:')
for p in pairs:
    print(p)
print('********************')

unique_tickers = pd.concat([filtered['seriesX'].iloc[1:], filtered['seriesY'].iloc[1:]]).drop_duplicates()
print('Unique tickers:')
for ut in unique_tickers:
    print(ut)
print('********************')

# Preview the source data
file_key = 'df_full_test.csv'
df_full = pd.read_csv(file_key).drop(['Unnamed: 0'], axis=1)
# obj = s3.get_object(Bucket=bucket_name, Key=file_key)
# df_full = pd.read_csv(StringIO(obj['Body'].read().decode('utf-8'))).drop(['Unnamed: 0'], axis=1)
display(df_full.sample(5))

********************
Trading pairs:
META DAXEX_ETF
AMZN BNBUSDT
AMZN TRXUSDT
SPY_ETF AVAXUSDT
SPY_ETF ETHUSDT
EQQQ_ETF BNBUSDT
BNBUSDT ADAUSDT
BNBUSDT BTCUSDT
BNBUSDT TRXUSDT
BNBUSDT XRPUSDT
********************
Unique tickers:
META
AMZN
SPY_ETF
EQQQ_ETF
BNBUSDT
DAXEX_ETF
TRXUSDT
AVAXUSDT
ETHUSDT
ADAUSDT
BTCUSDT
XRPUSDT
********************


,OpenTime,Open,High,Low,Close,Symbol
8034,2025-01-29 13:40:00,603.16500,603.50000,602.50100,602.76000,SPY_ETF
12021,2025-01-21 00:50:00,0.98560,0.98890,0.98510,0.98830,ADAUSDT
34165,2025-01-28 17:20:00,0.32289,0.32294,0.32149,0.32246,DOGEUSDT
20225,2025-01-23 21:10:00,0.34928,0.34944,0.34813,0.34849,DOGEUSDT
27855,2025-01-26 12:45:00,0.35170,0.35196,0.35140,0.35146,DOGEUSDT


In [6]:
for symbol in df_full.Symbol.unique():
    df_symbol = df_full[df_full['Symbol']==symbol]
    print(f"{symbol}: from {df_symbol['OpenTime'].min()} to {df_symbol['OpenTime'].max()}")

NKY225_ETF: from 2025-01-20 22:00:00 to 2025-02-03 04:15:00
CAC40_ETF: from 2025-01-21 06:00:00 to 2025-01-31 14:30:00
DAXEX_ETF: from 2025-01-21 06:00:00 to 2025-01-31 14:35:00
EQQQ_ETF: from 2025-01-21 06:00:00 to 2025-01-31 14:35:00
FTSE_ETF: from 2025-01-21 06:00:00 to 2025-01-31 14:35:00
AAPL: from 2025-01-21 12:30:00 to 2025-01-31 18:55:00
AMZN: from 2025-01-21 12:30:00 to 2025-01-31 18:55:00
BRK.B: from 2025-01-21 12:30:00 to 2025-01-31 18:55:00
DJIA_ETF: from 2025-01-21 12:30:00 to 2025-01-31 18:55:00
GOOGL: from 2025-01-21 12:30:00 to 2025-01-31 18:55:00
META: from 2025-01-21 12:30:00 to 2025-01-31 18:55:00
MSFT: from 2025-01-21 12:30:00 to 2025-01-31 18:55:00
NVDA: from 2025-01-21 12:30:00 to 2025-01-31 18:55:00
SPY_ETF: from 2025-01-21 12:30:00 to 2025-01-31 18:55:00
TSLA: from 2025-01-21 12:30:00 to 2025-01-31 18:55:00
AVGO: from 2025-01-23 12:30:00 to 2025-01-31 18:55:00
GOOG: from 2025-01-23 12:30:00 to 2025-01-31 18:55:00
ADAUSDT: from 2025-01-20 20:00:00 to 2025-01-30 2

# Features Engineering

## Create and normalizing features
1. Create price data from pairs data
2. Create features from pairs data
3. Load saved scaler
4. Perform normalization / standardization

In [7]:
scaler_type = "StandardScaler"
dict_data['df_close'] = {}
dict_data['df_feat'] = {}
dict_data['df_norm'] = {}
dict_data['scaler'] = {}

i = 1

for p in pairs:

    print(f"({i}) Pairs: {p}")
    tickerX, tickerY = p.split(' ')

    # 1. Create price from pairs data
    df_close = data.process_pairs_series(
        tickerX, 
        tickerY,
        df_full[df_full['Symbol']==tickerX],
        df_full[df_full['Symbol']==tickerY],
        "Close",
    )
    
    # 1. Create features from pairs data
    df_feat = feat.create_features(
        tickerX, 
        tickerY,
        df_full[df_full['Symbol']==tickerX],
        df_full[df_full['Symbol']==tickerY],
        config=None, # using default config in the FeaturesEngineering
        dropna=True
    )

    # 2. Load saved scaler
    scaler_name = f"{scaler_type.lower()}_{p.replace(' ', '_')}.pkl"
    dir_scaler = os.path.join(dir_models_scaler, scaler_name)
    scaler = pypickle.load(dir_scaler)

    # 3. Perform normalization / standardization
    data_scaled = scaler.transform(df_feat)
    df_norm = pd.DataFrame(
        data_scaled, index=df_feat.index, columns=df_feat.columns
    )
    df_norm = df_norm.rename(columns={'Spread':'NormalizedSpread'})

    dict_data['df_close'][p] = df_close
    dict_data['df_feat'][p] = df_feat
    dict_data['df_norm'][p] = df_norm
    dict_data['scaler'][p] = scaler

    i+=1
    print('')

(1) Pairs: META DAXEX_ETF
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_META_DAXEX_ETF.pkl]

(2) Pairs: AMZN BNBUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_AMZN_BNBUSDT.pkl]

(3) Pairs: AMZN TRXUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_AMZN_TRXUSDT.pkl]

(4) Pairs: SPY_ETF AVAXUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_SPY_ETF_AVAXUSDT.pkl]

(5) Pairs: SPY_ETF ETHUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_SPY_ETF_ETHUSDT.pkl]

(6) Pairs: EQQQ_ETF BNBUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_EQQQ_ETF_BNBUSDT.pkl]

(7) Pairs: BNBUSDT ADAUSDT
[pypickle] Pickle file loaded: [/ho

## Read LassoCV selected features

In [8]:
lasso_cv_name = f"lasso_cv_features.pkl"
dir_lasso = os.path.join(dir_models, lasso_cv_name)
dict_cv_features = pypickle.load(dir_lasso)

for key in dict_cv_features:
    print(f"{key}: {dict_cv_features[key]}")

[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/lasso_cv_features.pkl]
META DAXEX_ETF: ['META_EMA8', 'META_EMA55', 'META_RSI14', 'META_ATR14', 'META_StochD3', 'META_CCI20', 'META_WILLR14', 'DAXEX_ETF_EMA8', 'DAXEX_ETF_EMA21', 'DAXEX_ETF_MACD', 'DAXEX_ETF_BBupper', 'DAXEX_ETF_BBmiddle', 'DAXEX_ETF_BBlower', 'DAXEX_ETF_StochD3']
AMZN BNBUSDT: ['AMZN_EMA8', 'AMZN_EMA21', 'AMZN_EMA55', 'AMZN_RSI14', 'AMZN_ATR14', 'AMZN_StochD3', 'AMZN_WILLR14', 'BNBUSDT_EMA8', 'BNBUSDT_EMA55', 'BNBUSDT_MACD', 'BNBUSDT_RSI14', 'BNBUSDT_ATR14', 'BNBUSDT_StochD3', 'BNBUSDT_CCI20', 'BNBUSDT_WILLR14']
AMZN TRXUSDT: ['AMZN_EMA8', 'AMZN_EMA55', 'AMZN_RSI14', 'AMZN_ATR14', 'AMZN_StochD3', 'AMZN_CCI20', 'AMZN_WILLR14', 'TRXUSDT_MACD']
SPY_ETF AVAXUSDT: ['SPY_ETF_EMA8', 'SPY_ETF_EMA55', 'SPY_ETF_MACD', 'SPY_ETF_RSI14', 'SPY_ETF_BBupper', 'SPY_ETF_BBmiddle', 'SPY_ETF_BBlower', 'SPY_ETF_ATR14', 'SPY_ETF_StochK14', 'SPY_ETF_StochD3', 'SPY_ETF_CCI20', 'SPY_ETF_WILLR14', 'AVAXUSDT_E

## Reading the trained models hyperparameters

In [11]:
dict_data['hyperparameters'] = {}

dict_base = {
    'ridge': models.ridge_regression,
    'xgb': models.xgboost_regression,
    'rnn': models.rnn_regression,
    'lstm': models.lstm_regression,
    'transformer': models.transformer_regression,
}

for algo in dict_base.keys():
    print(f"\nAlgorithm: {algo}")
    df_hyper = pd.read_csv(os.path.join(dir_models_best, algo, "best_model_results.csv"))
    dict_data['hyperparameters'][algo] = df_hyper
    display(df_hyper)
    


Algorithm: ridge


,pair,best_mse,data_points,features,alpha,solver
0,META DAXEX_ETF,0.002754,1145,15,1.000,sparse_cg
1,AMZN BNBUSDT,0.001153,4991,16,0.001,lsqr
2,AMZN TRXUSDT,0.000376,4991,9,1.000,lsqr
3,SPY_ETF AVAXUSDT,0.007090,4997,21,0.010,sparse_cg
4,SPY_ETF ETHUSDT,0.000871,4997,16,0.001,svd
5,EQQQ_ETF BNBUSDT,0.000715,3217,13,0.001,sparse_cg
6,BNBUSDT ADAUSDT,0.000149,31051,9,0.010,sag
7,BNBUSDT BTCUSDT,0.000093,31051,9,0.001,svd
8,BNBUSDT TRXUSDT,0.000148,31051,12,0.100,sag
9,BNBUSDT XRPUSDT,0.000150,31051,12,1.000,sag



Algorithm: xgb


,pair,best_mse,data_points,features,learning_rate,n_estimators,max_depth
0,META DAXEX_ETF,0.026589,1145,15,0.05,50,7
1,AMZN BNBUSDT,0.006577,4991,16,0.10,200,5
2,AMZN TRXUSDT,0.000608,4991,9,0.05,200,5
3,SPY_ETF AVAXUSDT,0.323226,4997,21,0.05,200,7
4,SPY_ETF ETHUSDT,0.003511,4997,16,0.10,200,5
5,EQQQ_ETF BNBUSDT,0.028995,3217,13,0.10,200,5
6,BNBUSDT ADAUSDT,0.000159,31051,9,0.10,200,5
7,BNBUSDT BTCUSDT,0.000149,31051,9,0.10,200,9
8,BNBUSDT TRXUSDT,0.000135,31051,12,0.10,200,5
9,BNBUSDT XRPUSDT,0.000108,31051,12,0.10,200,5



Algorithm: rnn


,pair,best_mse,data_points,features,lookback,dropout_rate,batch_size
0,META DAXEX_ETF,0.021400,1145,15,15,0.2,64
1,AMZN BNBUSDT,0.003132,4991,16,10,0.2,32
2,AMZN TRXUSDT,0.001689,4991,9,15,0.2,32
3,SPY_ETF AVAXUSDT,0.028558,4997,21,5,0.3,32
4,SPY_ETF ETHUSDT,0.002134,4997,16,5,0.3,32
5,EQQQ_ETF BNBUSDT,0.004053,3217,13,10,0.2,64
6,BNBUSDT ADAUSDT,0.000362,31051,9,5,0.2,64
7,BNBUSDT BTCUSDT,0.000374,31051,9,10,0.2,64
8,BNBUSDT TRXUSDT,0.000388,31051,12,10,0.2,32
9,BNBUSDT XRPUSDT,0.000403,31051,12,15,0.2,64



Algorithm: lstm


,pair,best_mse,data_points,features,units,lookback,dropout_rate,batch_size
0,META DAXEX_ETF,0.018882,1145,15,64,5,0.25,32
1,AMZN BNBUSDT,0.002946,4991,16,256,10,0.25,32
2,AMZN TRXUSDT,0.001328,4991,9,256,15,0.25,32
3,SPY_ETF AVAXUSDT,0.025010,4997,21,256,10,0.25,32
4,SPY_ETF ETHUSDT,0.001900,4997,16,128,15,0.25,32
5,EQQQ_ETF BNBUSDT,0.003296,3217,13,256,15,0.25,64
6,BNBUSDT ADAUSDT,0.000428,31051,9,64,15,0.25,64
7,BNBUSDT BTCUSDT,0.000305,31051,9,256,5,0.25,32
8,BNBUSDT TRXUSDT,0.000462,31051,12,64,15,0.25,32
9,BNBUSDT XRPUSDT,0.000393,31051,12,128,15,0.25,32



Algorithm: transformer


,pair,best_mse,data_points,features,num_heads,lookback,dropout_rate,batch_size,ff_dim
0,META DAXEX_ETF,0.030082,1145,15,6,5,0.25,32,128
1,AMZN BNBUSDT,0.011686,4991,16,2,15,0.25,64,128
2,AMZN TRXUSDT,0.002809,4991,9,2,10,0.25,32,128
3,SPY_ETF AVAXUSDT,0.036903,4997,21,4,5,0.25,64,64
4,SPY_ETF ETHUSDT,0.006406,4997,16,2,5,0.25,64,128
5,EQQQ_ETF BNBUSDT,0.023795,3217,13,4,5,0.25,32,128
6,BNBUSDT ADAUSDT,0.001136,31051,9,4,15,0.25,32,64
7,BNBUSDT BTCUSDT,0.002063,31051,9,2,5,0.25,32,64
8,BNBUSDT TRXUSDT,0.001045,31051,12,6,15,0.25,32,64
9,BNBUSDT XRPUSDT,0.001856,31051,12,6,10,0.25,32,64


## Reading the trained best model from hyperparameters training

In [38]:
i = 1
verbose = True
list_results = []

for p in pairs.iloc[0:1]:
    print(f"({i}) Pairs: {p}")

    # 1. Read normalized features
    df_norm = dict_data['df_norm'][p]
    print(f'# of original features: {len(df_norm.columns)-1}')
    
    # 2. Create X, y based on LassoCV
    lasso_features_cols = dict_cv_features[p]
    target_col = df_norm.columns.tolist()[-1]
    df_lasso = pd.concat([df_norm[lasso_features_cols], df_norm[target_col]], axis=1)
    print(f'# of features after LassoCV: {len(df_lasso.columns)-1}')

    # 3. Load train models and running the trainning process
    ridge, ridge_mse, ridge_test_df, ridge_time, ridge_memory = models.ridge_regression(
        df=df_lasso, 
        p=p,
        pickle_file=os.path.join(dir_models_best, 'ridge', f"best_ridge_{p.replace(' ', '_')}.pkl"),
        testing=True,
        verbose=verbose,
    )
    
    xgb, xgb_mse, xgb_test_df, xgb_time, xgb_memory = models.xgboost_regression(
        df=df_lasso, 
        p=p,
        pickle_file=os.path.join(dir_models_best, 'xgb', f"best_xgb_{p.replace(' ', '_')}.pkl"),
        testing=True,
        verbose=verbose,
    )
    
    lstm, lstm_mse, lstm_test_df, lstm_time, lstm_memory = models.lstm_regression(
        df=df_lasso, 
        p=p,
        pickle_file=os.path.join(dir_models_best, 'lstm', f"best_lstm_{p.replace(' ', '_')}.pkl"),
        testing=True,
        verbose=verbose,
    )

    rnn, rnn_mse, rnn_test_df, rnn_time, rnn_memory = models.rnn_regression(
        df=df_lasso, 
        p=p,
        pickle_file=os.path.join(dir_models_best, 'rnn', f"best_rnn_{p.replace(' ', '_')}.pkl"),
        testing=True,
        verbose=verbose,
    )
    
    trf, trf_mse, trf_test_df, trf_time, trf_memory = models.transformer_regression(
        df=df_lasso, 
        p=p,
        pickle_file=os.path.join(dir_models_best, 'transformer', f"best_transformer_{p.replace(' ', '_')}.pkl"),
        testing=True,
        verbose=verbose,
    )

(1) Pairs: META DAXEX_ETF
# of original features: 26
# of features after LassoCV: 14
Testing Phase
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/best/ridge/best_ridge_META_DAXEX_ETF.pkl]
Ridge Regression MSE: 0.028219
Testing Phase
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/best/xgb/best_xgb_META_DAXEX_ETF.pkl]
Loaded XGBoost model from /home/alfred/projects/mscfe-masters-project/src/models/best/xgb/best_xgb_META_DAXEX_ETF.pkl.
XGBoost MSE: 3.645550292926379
Testing Phase
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/best/lstm/best_lstm_META_DAXEX_ETF.pkl]
Loaded LSTM model from /home/alfred/projects/mscfe-masters-project/src/models/best/lstm/best_lstm_META_DAXEX_ETF.pkl.
1/5 ━━━━━━━━━━━━━━━━━━━━ 1s 375ms/step

I0000 00:00:1740823541.960542   22651 service.cc:148] XLA service 0xd660980 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740823541.960563   22651 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-03-01 18:05:41.974836: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1740823541.987344   22651 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1740823542.084977   22651 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step 
LSTM MSE: 2.0232839209213824
Failed to run LSTM model: cannot access local variable 'test_size' where it is not associated with a value
Testing Phase
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/best/rnn/best_rnn_META_DAXEX_ETF.pkl]
Loaded RNN model from /home/alfred/projects/mscfe-masters-project/src/models/best/rnn/best_rnn_META_DAXEX_ETF.pkl.
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step 
RNN MSE: 1.4940568505445857
Failed to run RNN model: cannot access local variable 'test_size' where it is not associated with a value
Testing Phase
Failed to run Transformer model: cannot access local variable 'X_train' where it is not associated with a value


In [32]:
ridge_mse

0.02821850640561618

In [12]:
df_temp = (df_full
           .groupby(by=['Symbol', 'OpenTime'])
           .agg({'Open': 'first',
                 'Close': 'first'})
           .reset_index())
df_temp['OpenTime'] = pd.to_datetime(df_temp['OpenTime'])
df_temp = df_temp.sort_values(by=['Symbol', 'OpenTime'])
df_temp = (df_temp
           .groupby(by=['Symbol'], group_keys=False)
           # Shift open prices within each symbol
           .apply(lambda x: x.assign(NextOpen=x['Open'].shift(1)))
           .reset_index(level=0, drop=True))

/tmp/ipykernel_19629/848331043.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(NextOpen=x['Open'].shift(1)))


In [13]:
df_temp

,Symbol,OpenTime,Open,Close,NextOpen
0,AAPL,2025-01-21 12:30:00,224.0000,222.3400,NaN
1,AAPL,2025-01-21 12:35:00,222.2950,222.3350,224.0000
2,AAPL,2025-01-21 12:40:00,222.3200,220.9150,222.2950
3,AAPL,2025-01-21 12:45:00,220.9100,222.3753,222.3200
4,AAPL,2025-01-21 12:50:00,222.3550,221.3250,220.9100
...,...,...,...,...,...
40246,XRPUSDT,2025-01-30 19:40:00,3.1404,3.1420,3.1434
40247,XRPUSDT,2025-01-30 19:45:00,3.1420,3.1360,3.1404
40248,XRPUSDT,2025-01-30 19:50:00,3.1360,3.1300,3.1420
40249,XRPUSDT,2025-01-30 19:55:00,3.1301,3.1276,3.1360


In [ ]:
df_test.PredictedSignal.value_counts()

In [ ]:
df_test